In [1]:
"""This code is used to clip the topo tif file in a bounding box that defines the grid 
July 20, 2024 written by Tarandeep Kalra
"""

'This code is used to clip the topo tif file in a bounding box that defines the grid \nJuly 20, 2024 written by Tarandeep Kalra\n'

In [2]:

import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from osgeo import gdal, osr, ogr
import rasterio
from rasterio.warp import calculate_default_transform, reproject
from rasterio.crs import CRS
from shapely.geometry import Polygon
#from scipy.io import loadmat          

In [3]:
#gridfile = r'C:\Users\tkalra\Desktop\NFWP_Swan_coral_USVirgin\69532_Coral_Reefs_St_Croix\Grid\output_new_small2.tif'
#out_grid_file = r'C:\Users\tkalra\Desktop\NFWP_Swan_coral_USVirgin\69532_Coral_Reefs_St_Croix\Grid\restored_USVI_rasterio.tif'
input = r'C:\Users\tkalra\Downloads\2018_usgs_southJersey_Job1056745\2018_usgs_southJersey_Job1056745.tif'
# Open the TIFF file
ds = gdal.Open(input)

In [4]:

# Define the coordinates for clipping (x, y)
#coords = [(100, 100), (200, 100), (200, 200), (100, 200), (50, 150), (150, 50)]
coords = [(336284, 142058), (335020, 150495), (352506, 150495), (336037, 144437), (349013, 141203)]

In [5]:
# Create a Polygon object from the coordinates
polygon = Polygon(coords)

In [6]:

# Open the input TIFF file using Rasterio
with rasterio.open(input) as src:
    # Get the CRS of the input TIFF file
    src_crs = src.crs

    # Calculate the default transform for the clipped output
    transform, width, height = calculate_default_transform(
        src_crs, src_crs, polygon.bounds[0], polygon.bounds[1], polygon.bounds[2], polygon.bounds[3], *src.shape)

    # Create a new Rasterio dataset for the clipped output
    with rasterio.open(
        'clipped.tif',
        'w',
        driver='GTiff',
        height=height,
        width=width,
        count=src.count,
        dtype=src.dtypes[0],
        crs=src_crs,
        transform=transform,
    ) as dst:
        # Clip the input TIFF file using the polygon
        reproject(
            source=rasterio.band(src, 1),
            destination=rasterio.band(dst, 1),
            src_transform=src.transform,
            src_crs=src_crs,
            dst_transform=transform,
            dst_crs=src_crs,
            resampling=rasterio.warp.Resampling.bilinear,
            num_threads=4,
        )